# Molecular Generator Evaluation using TUPOR, SESY and ASER Metrics

🔹 **Objective**  
This notebook evaluates molecular generators by computing four key metrics:  
   - **TUPOR**: scaffold recall metrics  
   - **SESY**: scaffold hopping potencial  
   - **ASER**: chemical space exploration

🔹 **Workflow**  
1️⃣ **Compute Metrics**: The script calculates TUPOR, SESY and ASER for different molecular generators.  
2️⃣ **Merge Data**: Results from multiple generators are combined into a single Pandas DataFrame.  
3️⃣ **Normalize Values**: The computed metrics are normalized using Min-Max scaling for comparison.  
4️⃣ **Save Outputs**: Processed data is stored in CSV files for further analysis.  

🔹 **Data Structure**  
- The calculations are performed for different **scaffold types** (`csk`, `murcko`) and **cluster types** (`dis`, `sim`).  
- Results are computed for multiple **generators** (`Molpher`, `REINVENT`, `DrugEx`, `GB_GA`, etc.).  
- The analysis is conducted for a specific **biological target receptor**, such as the **Glucocorticoid receptor**.

This notebook allows us to compare the performance of various molecular generators in terms of structural diversity, similarity to known bioactive compounds, and synthetic feasibility.

# Loading required libraries

In [1]:
from src import metrics # Importing custom metric functions
import importlib as imp
imp.reload(metrics)

<module 'src.metrics' from '/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/src/metrics.py'>

# Function to calculate metrics

In [2]:
def calculate_metrics(type_cluster, type_scaffold,generator, receptor, ncpus = 1):
    """
    Function to calculate molecular generation metrics.
    
    Parameters:
    - scaffold_type: Type of scaffold (e.g., 'csk' or 'murcko')
    - type_cluster: Cluster type  (e.g., 'dis' or 'sim') dis = Dissimilarity split; sim = Similarity split
    - generator: Name of the molecular generator
    - receptor: Target receptor for drug design

    Returns:
    - Computed metrics
    """
    mt = metrics.Metrics(type_cluster, type_scaffold, generator, receptor, ncpus)     
    result = mt.calculate_metrics()
    display(result)
    return result

# Define parameters for metric calculations

In [3]:
type_cluster = 'sim' #options: 'dis'|'sim' 
type_scaffold = 'csk' #options: 'csk'|'murcko'
generator = 'Molpher' #options: 'Molpher'|'DrugEx'|'REINVENT'|'addcarbon'
receptor = 'Leukocyte_elastase' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

calculate_metrics(type_cluster,type_scaffold,generator,receptor, ncpus = 10)

NUMBER:  0


[10:03:34] Explicit valence for atom # 12 C, 5, is greater than permitted
[10:03:49] Explicit valence for atom # 28 C, 5, is greater than permitted
[10:03:49] Explicit valence for atom # 28 C, 5, is greater than permitted
[10:03:49] Explicit valence for atom # 27 C, 5, is greater than permitted
[10:04:05] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:04:05] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:04:05] Explicit valence for atom # 7 C, 5, is greater than permitted
[10:04:05] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:04:05] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:04:05] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:04:05] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:04:05] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:04:05] Explicit valence for atom # 7 C, 5, is greater than permitted
[10:04:05] Explicit valence for atom # 7 C, 5, 

NUMBER:  1


[10:08:55] Explicit valence for atom # 24 C, 5, is greater than permitted
[10:08:55] Explicit valence for atom # 23 C, 5, is greater than permitted
[10:08:55] Explicit valence for atom # 23 C, 5, is greater than permitted
[10:08:55] Explicit valence for atom # 24 C, 5, is greater than permitted
[10:08:55] Explicit valence for atom # 23 C, 5, is greater than permitted
[10:09:48] Explicit valence for atom # 11 C, 5, is greater than permitted
[10:09:48] Explicit valence for atom # 11 C, 5, is greater than permitted
[10:09:48] Explicit valence for atom # 11 C, 5, is greater than permitted
[10:10:01] Explicit valence for atom # 2 C, 5, is greater than permitted
[10:10:01] Explicit valence for atom # 24 C, 5, is greater than permitted
[10:10:01] Explicit valence for atom # 24 C, 5, is greater than permitted
[10:10:07] Explicit valence for atom # 7 C, 5, is greater than permitted
[10:10:48] Explicit valence for atom # 15 C, 5, is greater than permitted
[10:12:09] Explicit valence for atom # 1

NUMBER:  2


...

[10:21:36] Explicit valence for atom # 10 C, 5, is greater than permitted
[10:21:54] Explicit valence for atom # 1 C, 5, is greater than permitted
[10:21:56] Explicit valence for atom # 24 C, 5, is greater than permitted
[10:21:56] Explicit valence for atom # 24 C, 5, is greater than permitted
[10:23:58] Explicit valence for atom # 26 C, 5, is greater than permitted
[10:23:58] Explicit valence for atom # 25 C, 5, is greater than permitted
[10:23:58] Explicit valence for atom # 25 C, 5, is greater than permitted
[10:24:35] Explicit valence for atom # 31 C, 5, is greater than permitted
[10:24:35] Explicit valence for atom # 26 C, 5, is greater than permitted
[10:24:35] Explicit valence for atom # 27 C, 5, is greater than permitted
[10:24:42] Explicit valence for atom # 29 C, 5, is greater than permitted
[10:25:23] Explicit valence for atom # 15 C, 5, is greater than permitted


NUMBER:  4


[10:27:38] Explicit valence for atom # 25 C, 5, is greater than permitted
[10:27:38] Explicit valence for atom # 24 C, 5, is greater than permitted
[10:27:38] Explicit valence for atom # 24 C, 5, is greater than permitted
[10:28:17] Explicit valence for atom # 15 C, 5, is greater than permitted
[10:28:17] Explicit valence for atom # 16 C, 5, is greater than permitted
[10:28:17] Explicit valence for atom # 15 C, 5, is greater than permitted
[10:28:17] Explicit valence for atom # 15 C, 5, is greater than permitted
[10:28:17] Explicit valence for atom # 15 C, 5, is greater than permitted
[10:28:17] Explicit valence for atom # 14 C, 5, is greater than permitted
[10:28:17] Explicit valence for atom # 15 C, 5, is greater than permitted
[10:28:17] Explicit valence for atom # 15 C, 5, is greater than permitted
[10:28:21] Explicit valence for atom # 21 C, 5, is greater than permitted
[10:28:21] Explicit valence for atom # 21 C, 5, is greater than permitted
[10:28:21] Explicit valence for atom #

,name,type_cluster,scaffold,SSo,TUPOR_,TUPOR,SESY,ASER
0,Molpher_0,sim,csk,1624700.0,43/46,0.934783,0.138021,0.031430
1,Molpher_1,sim,csk,1662823.0,38/44,0.863636,0.130204,0.009258
2,Molpher_2,sim,csk,1546835.0,36/42,0.857143,0.121280,0.007056
3,Molpher_3,sim,csk,1749302.0,32/42,0.761905,0.123391,0.011643
4,Molpher_4,sim,csk,1722174.0,32/42,0.761905,0.123599,0.003529
5,Molpher_mean,sim,csk,1661166.8,-,0.835874,0.127299,0.012583


,name,type_cluster,scaffold,SSo,TUPOR_,TUPOR,SESY,ASER
0,Molpher_0,sim,csk,1624700.0,43/46,0.934783,0.138021,0.031430
1,Molpher_1,sim,csk,1662823.0,38/44,0.863636,0.130204,0.009258
2,Molpher_2,sim,csk,1546835.0,36/42,0.857143,0.121280,0.007056
3,Molpher_3,sim,csk,1749302.0,32/42,0.761905,0.123391,0.011643
4,Molpher_4,sim,csk,1722174.0,32/42,0.761905,0.123599,0.003529
5,Molpher_mean,sim,csk,1661166.8,-,0.835874,0.127299,0.012583


# Execute metric calculation function

In [ ]:
for receptor in ['Glucocorticoid_receptor', 'Leukocyte_elastase']:
    for type_scaffold in ['csk', 'murcko']:
        for type_cluster in ['dis','sim']:
            for subset in ['','_500k', '_250k', '_125k', '_62.5k']:
                ncpus = 10
                
                # Define generator names with different epsilon values
                generators_name_list = [
                    f"Molpher{subset}",
                    f"REINVENT{subset}",
                    f"DrugEx_GT_epsilon_0.1{subset}",
                    f"DrugEx_GT_epsilon_0.6{subset}",
                    f"DrugEx_RNN_epsilon_0.1{subset}",
                    f"DrugEx_RNN_epsilon_0.6{subset}",
                    f"GB_GA_mut_r_0.01{subset}",
                    f"GB_GA_mut_r_0.5{subset}",
                    f"addcarbon{subset}"
                ]
                for generator in generators_name_list:
                    calculate_metrics(type_cluster,type_scaffold,generator,receptor,ncpus = ncpus)

NUMBER:  0


[10:47:50] Explicit valence for atom # 18 C, 5, is greater than permitted
[10:48:08] Explicit valence for atom # 5 C, 5, is greater than permitted
[10:48:08] Explicit valence for atom # 5 C, 5, is greater than permitted
[10:48:08] Explicit valence for atom # 7 C, 5, is greater than permitted
[10:48:08] Explicit valence for atom # 22 C, 5, is greater than permitted


NUMBER:  1
NUMBER:  2


[10:54:50] Explicit valence for atom # 6 C, 5, is greater than permitted
[10:55:07] Explicit valence for atom # 9 C, 5, is greater than permitted
[10:55:08] Explicit valence for atom # 12 C, 5, is greater than permitted
[10:55:08] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:55:08] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:55:08] Explicit valence for atom # 3 C, 5, is greater than permitted
[10:55:08] Explicit valence for atom # 4 C, 5, is greater than permitted
[10:55:19] Explicit valence for atom # 29 C, 5, is greater than permitted
[10:55:19] Explicit valence for atom # 29 C, 5, is greater than permitted


NUMBER:  3


...

,name,type_cluster,scaffold,SSo,TUPOR_,TUPOR,SESY,ASER
0,Molpher_0,dis,csk,916546.0,15/40,0.375000,0.114250,0.003291
1,Molpher_1,dis,csk,1070030.0,15/23,0.652174,0.111263,0.001106
2,Molpher_2,dis,csk,906891.0,9/40,0.225000,0.116045,0.000114
3,Molpher_3,dis,csk,693460.0,20/37,0.540540,0.129346,0.006726
4,Molpher_4,dis,csk,532962.0,34/43,0.790698,0.153170,0.011708
5,Molpher_mean,dis,csk,823977.8,-,0.516682,0.124815,0.004589


NUMBER:  0


[11:01:08] Explicit valence for atom # 36 C greater than permitted
[11:01:19] Explicit valence for atom # 36 C, 5, is greater than permitted
[11:01:24] Explicit valence for atom # 35 C greater than permitted
[11:01:42] Explicit valence for atom # 42 C greater than permitted
[11:01:45] Explicit valence for atom # 38 C, 5, is greater than permitted
[11:01:51] Explicit valence for atom # 37 C, 5, is greater than permitted
[11:01:51] Explicit valence for atom # 40 C greater than permitted
[11:01:53] Explicit valence for atom # 2 C greater than permitted
[11:01:56] Explicit valence for atom # 2 C, 5, is greater than permitted
[11:02:01] Explicit valence for atom # 37 C greater than permitted
[11:02:01] Explicit valence for atom # 34 C greater than permitted
[11:02:12] Explicit valence for atom # 35 C greater than permitted
[11:02:12] Explicit valence for atom # 36 C, 5, is greater than permitted
[11:02:15] Explicit valence for atom # 41 C greater than permitted
[11:02:16] Explicit valence f

NUMBER:  1


[11:03:52] Explicit valence for atom # 31 C greater than permitted
[11:03:58] Explicit valence for atom # 7 C greater than permitted
[11:04:09] Explicit valence for atom # 6 C greater than permitted
[11:04:10] Explicit valence for atom # 3 C greater than permitted
[11:04:17] Explicit valence for atom # 27 C greater than permitted
[11:04:24] Explicit valence for atom # 4 C greater than permitted
[11:04:30] Explicit valence for atom # 24 C greater than permitted
[11:04:41] Explicit valence for atom # 23 C, 5, is greater than permitted
[11:04:41] Explicit valence for atom # 2 C greater than permitted
[11:04:47] Explicit valence for atom # 10 C greater than permitted
[11:04:52] Explicit valence for atom # 3 C greater than permitted
[11:04:57] Explicit valence for atom # 5 C, 5, is greater than permitted
[11:04:58] Explicit valence for atom # 8 C, 5, is greater than permitted
[11:05:01] Explicit valence for atom # 26 C greater than permitted
[11:05:03] Explicit valence for atom # 2 C greate

## Combining and Normalizing Metrics

The following cell runs functions that:

- merge the mean values of all metrics into a single `pandas.DataFrame` (using `connect_mean_value`)
- apply Min-Max normalization to scale the values (using `connect_mean_value_normalized`)


In [26]:
from src import metrics_connection # Importing custom metric functions
imp.reload(metrics_connection)

<module 'src.metrics_connection' from '/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/src/metrics_connection.py'>

In [28]:
for receptor in ['Glucocorticoid_receptor', 'Leukocyte_elastase']:
    for type_scaffold in ['csk', 'murcko']:
        for type_cluster in ['dis', 'sim']:  # Different cluster types
            for subset in ['_62.5k']:
            
                # Define generator names with different epsilon values
                generators_name_list = [
                    f"Molpher{subset}",
                    f"REINVENT{subset}",
                    f"DrugEx_GT_epsilon_0.1{subset}",
                    f"DrugEx_GT_epsilon_0.6{subset}",
                    f"DrugEx_RNN_epsilon_0.1{subset}",
                    f"DrugEx_RNN_epsilon_0.6{subset}",
                    f"GB_GA_mut_r_0.01{subset}",
                    f"GB_GA_mut_r_0.5{subset}",
                    f"addcarbon{subset}"
                ]
    
                # Connect and process mean values
                df = metrics_connection.connect_mean_value(type_cluster, type_scaffold, generators_name_list, receptor, subset)
                df1 = metrics_connection.connect_mean_value_normalized(type_cluster, type_scaffold, generators_name_list, receptor, subset)
                
                display(df)
                display(df1)

,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,SESY,ASER,CwASo
0,Molpher_62.5k_mean,dis,csk,11998.6,64267.4,-,0.275797,0.186985,0.005181,322.0
1,REINVENT_62.5k_mean,dis,csk,15357.6,61685.2,-,0.210779,0.249357,0.004440,268.2
2,DrugEx_GT_epsilon_0.1_62.5k_mean,dis,csk,19708.4,62449.4,-,0.345967,0.315596,0.004801,297.4
3,DrugEx_GT_epsilon_0.6_62.5k_mean,dis,csk,28496.8,62459.4,-,0.532874,0.456241,0.007952,490.6
4,DrugEx_RNN_epsilon_0.1_62.5k_mean,dis,csk,12740.4,62468.0,-,0.230480,0.203956,0.005124,315.2
5,DrugEx_RNN_epsilon_0.6_62.5k_mean,dis,csk,12328.4,61886.6,-,0.480158,0.199214,0.018424,1085.2
6,GB_GA_mut_r_0.01_62.5k_mean,dis,csk,23549.6,62496.0,-,0.326206,0.376818,0.003068,190.6
7,GB_GA_mut_r_0.5_62.5k_mean,dis,csk,17976.2,62498.0,-,0.315395,0.287629,0.004304,267.0
8,addcarbon_62.5k_mean,dis,csk,NaN,62500.0,-,0.033403,0.013565,0.001968,NaN


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,SESY,ASER,CwASo
0,Molpher_62.5k_mean,dis,csk,0.000000,1.000000,-,0.485302,0.391754,0.195269,0.146881
1,REINVENT_62.5k_mean,dis,csk,0.203598,0.000000,-,0.355127,0.532651,0.150208,0.086743
2,DrugEx_GT_epsilon_0.1_62.5k_mean,dis,csk,0.467312,0.295949,-,0.625790,0.682286,0.172158,0.119383
3,DrugEx_GT_epsilon_0.6_62.5k_mean,dis,csk,1.000000,0.299822,-,1.000000,1.000000,0.363616,0.335345
4,DrugEx_RNN_epsilon_0.1_62.5k_mean,dis,csk,0.044962,0.303152,-,0.394571,0.430092,0.191762,0.139280
5,DrugEx_RNN_epsilon_0.6_62.5k_mean,dis,csk,0.019990,0.077996,-,0.894458,0.419379,1.000000,1.000000
6,GB_GA_mut_r_0.01_62.5k_mean,dis,csk,0.700137,0.313996,-,0.586226,0.820584,0.066858,0.000000
7,GB_GA_mut_r_0.5_62.5k_mean,dis,csk,0.362318,0.314770,-,0.564581,0.619107,0.141932,0.085401
8,addcarbon_62.5k_mean,dis,csk,NaN,0.315545,-,0.000000,0.000000,0.000000,NaN


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,SESY,ASER,CwASo
0,Molpher_62.5k_mean,sim,csk,11481.0,62631.6,-,0.466779,0.183303,0.014613,900.0
1,REINVENT_62.5k_mean,sim,csk,17606.4,61914.6,-,0.303971,0.284446,0.008965,548.6
2,DrugEx_GT_epsilon_0.1_62.5k_mean,sim,csk,19859.4,62444.0,-,0.375883,0.318032,0.013917,842.6
3,DrugEx_GT_epsilon_0.6_62.5k_mean,sim,csk,28863.8,62467.2,-,0.720447,0.462054,0.017285,1058.0
4,DrugEx_RNN_epsilon_0.1_62.5k_mean,sim,csk,13718.4,62445.8,-,0.278013,0.219666,0.020624,1255.6
5,DrugEx_RNN_epsilon_0.6_62.5k_mean,sim,csk,12482.8,61978.6,-,0.654225,0.201373,0.026403,1583.2
6,GB_GA_mut_r_0.01_62.5k_mean,sim,csk,23878.4,62497.6,-,0.421264,0.382069,0.006345,393.8
7,GB_GA_mut_r_0.5_62.5k_mean,sim,csk,18052.4,62497.4,-,0.431339,0.288850,0.009972,616.0
8,addcarbon_62.5k_mean,sim,csk,855.2,62500.0,-,0.201548,0.013683,0.014452,884.4


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,SESY,ASER,CwASo
0,Molpher_62.5k_mean,sim,csk,0.379376,1.000000,-,0.511141,0.378303,0.412193,0.425593
1,REINVENT_62.5k_mean,sim,csk,0.598073,0.000000,-,0.197384,0.603883,0.130586,0.130150
2,DrugEx_GT_epsilon_0.1_62.5k_mean,sim,csk,0.678513,0.738354,-,0.335970,0.678790,0.377468,0.377333
3,DrugEx_GT_epsilon_0.6_62.5k_mean,sim,csk,1.000000,0.770711,-,1.000000,1.000000,0.545401,0.558433
4,DrugEx_RNN_epsilon_0.1_62.5k_mean,sim,csk,0.459259,0.740865,-,0.147358,0.459403,0.711857,0.724567
5,DrugEx_RNN_epsilon_0.6_62.5k_mean,sim,csk,0.415144,0.089261,-,0.872379,0.418604,1.000000,1.000000
6,GB_GA_mut_r_0.01_62.5k_mean,sim,csk,0.822005,0.813110,-,0.423427,0.821610,0.000000,0.000000
7,GB_GA_mut_r_0.5_62.5k_mean,sim,csk,0.613997,0.812831,-,0.442843,0.613704,0.180795,0.186817
8,addcarbon_62.5k_mean,sim,csk,0.000000,0.816457,-,0.000000,0.000000,0.404141,0.412477


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,SESY,ASER,CwASo
0,Molpher_62.5k_mean,dis,murcko,20039.0,64267.4,-,0.057369,0.312280,0.001007,63.0
1,REINVENT_62.5k_mean,dis,murcko,48606.4,61693.0,-,0.038883,0.787626,0.000608,36.8
2,DrugEx_GT_epsilon_0.1_62.5k_mean,dis,murcko,41494.2,62451.4,-,0.075826,0.664439,0.001079,67.2
3,DrugEx_GT_epsilon_0.6_62.5k_mean,dis,murcko,47137.4,62464.8,-,0.151764,0.754621,0.001773,110.4
4,DrugEx_RNN_epsilon_0.1_62.5k_mean,dis,murcko,37399.6,62488.6,-,0.025324,0.598504,0.000615,38.4
5,DrugEx_RNN_epsilon_0.6_62.5k_mean,dis,murcko,37169.6,61917.2,-,0.155704,0.600296,0.001597,98.6
6,GB_GA_mut_r_0.01_62.5k_mean,dis,murcko,40663.6,62496.0,-,0.038619,0.650659,0.000311,19.4
7,GB_GA_mut_r_0.5_62.5k_mean,dis,murcko,34164.4,62498.6,-,0.044642,0.546643,0.000336,21.0
8,addcarbon_62.5k_mean,dis,murcko,2873.6,62500.0,-,0.015227,0.045978,0.000805,50.2


...

,name,type_cluster,scaffold,SSo,TUPOR_,TUPOR,SESY,ASER
0,Molpher_62.5k_mean,sim,csk,1.000000,-,0.735996,0.464484,0.027487
1,REINVENT_62.5k_mean,sim,csk,0.925972,-,0.293472,0.565312,0.000000
2,DrugEx_GT_epsilon_0.1_62.5k_mean,sim,csk,0.981123,-,0.685674,0.815493,0.189792
3,DrugEx_GT_epsilon_0.6_62.5k_mean,sim,csk,0.959769,-,1.000000,1.000000,0.296499
4,DrugEx_RNN_epsilon_0.1_62.5k_mean,sim,csk,0.851058,-,0.377163,0.345306,1.000000
5,DrugEx_RNN_epsilon_0.6_62.5k_mean,sim,csk,0.895171,-,0.997812,0.498345,0.676083
6,GB_GA_mut_r_0.01_62.5k_mean,sim,csk,0.480458,-,0.000000,0.423683,0.116545
7,GB_GA_mut_r_0.5_62.5k_mean,sim,csk,0.000000,-,0.143040,0.337055,0.306286
8,addcarbon_62.5k_mean,sim,csk,0.986966,-,0.166456,0.000000,0.040378


,name,type_cluster,scaffold,SSo,TUPOR_,TUPOR,SESY,ASER
0,Molpher_62.5k_mean,dis,murcko,62588.2,-,0.096729,0.359111,0.001840
1,REINVENT_62.5k_mean,dis,murcko,61703.0,-,0.122112,0.786161,0.000612
2,DrugEx_GT_epsilon_0.1_62.5k_mean,dis,murcko,62423.8,-,0.122074,0.748953,0.007241
3,DrugEx_GT_epsilon_0.6_62.5k_mean,dis,murcko,62219.2,-,0.288677,0.731255,0.006152
4,DrugEx_RNN_epsilon_0.1_62.5k_mean,dis,murcko,61682.6,-,0.059928,0.577273,0.015026
5,DrugEx_RNN_epsilon_0.6_62.5k_mean,dis,murcko,61508.8,-,0.181870,0.635373,0.009665
6,GB_GA_mut_r_0.01_62.5k_mean,dis,murcko,55988.2,-,0.045180,0.373767,0.007213
7,GB_GA_mut_r_0.5_62.5k_mean,dis,murcko,50724.8,-,0.037591,0.364277,0.006057
8,addcarbon_62.5k_mean,dis,murcko,62500.0,-,0.029286,0.075600,0.000339


,name,type_cluster,scaffold,SSo,TUPOR_,TUPOR,SESY,ASER
0,Molpher_62.5k_mean,dis,murcko,1.000000,-,0.260004,0.398996,0.102201
1,REINVENT_62.5k_mean,dis,murcko,0.925384,-,0.357860,1.000000,0.018575
2,DrugEx_GT_epsilon_0.1_62.5k_mean,dis,murcko,0.986142,-,0.357715,0.947636,0.469949
3,DrugEx_GT_epsilon_0.6_62.5k_mean,dis,murcko,0.968896,-,1.000000,0.922729,0.395793
4,DrugEx_RNN_epsilon_0.1_62.5k_mean,dis,murcko,0.923664,-,0.118133,0.706024,1.000000
5,DrugEx_RNN_epsilon_0.6_62.5k_mean,dis,murcko,0.909014,-,0.588240,0.787790,0.634983
6,GB_GA_mut_r_0.01_62.5k_mean,dis,murcko,0.443667,-,0.061276,0.419621,0.468002
7,GB_GA_mut_r_0.5_62.5k_mean,dis,murcko,0.000000,-,0.032019,0.406266,0.389277
8,addcarbon_62.5k_mean,dis,murcko,0.992565,-,0.000000,0.000000,0.000000


,name,type_cluster,scaffold,SSo,TUPOR_,TUPOR,SESY,ASER
0,Molpher_62.5k_mean,sim,murcko,62641.2,-,0.196671,0.358739,0.004471
1,REINVENT_62.5k_mean,sim,murcko,61846.0,-,0.101651,0.814765,0.001225
2,DrugEx_GT_epsilon_0.1_62.5k_mean,sim,murcko,62440.2,-,0.236030,0.740815,0.005546
3,DrugEx_GT_epsilon_0.6_62.5k_mean,sim,murcko,62215.4,-,0.409863,0.755204,0.008470
4,DrugEx_RNN_epsilon_0.1_62.5k_mean,sim,murcko,61114.2,-,0.062564,0.582847,0.019449
5,DrugEx_RNN_epsilon_0.6_62.5k_mean,sim,murcko,61618.6,-,0.303905,0.661860,0.012136
6,GB_GA_mut_r_0.01_62.5k_mean,sim,murcko,57020.8,-,0.042138,0.408830,0.003466
7,GB_GA_mut_r_0.5_62.5k_mean,sim,murcko,51823.4,-,0.043280,0.388567,0.005038
8,addcarbon_62.5k_mean,sim,murcko,62500.0,-,0.114764,0.075520,0.001780


,name,type_cluster,scaffold,SSo,TUPOR_,TUPOR,SESY,ASER
0,Molpher_62.5k_mean,sim,murcko,1.000000,-,0.420241,0.383120,0.178091
1,REINVENT_62.5k_mean,sim,murcko,0.926492,-,0.161841,1.000000,0.000000
2,DrugEx_GT_epsilon_0.1_62.5k_mean,sim,murcko,0.981420,-,0.527273,0.899966,0.237100
3,DrugEx_GT_epsilon_0.6_62.5k_mean,sim,murcko,0.960639,-,1.000000,0.919431,0.397533
4,DrugEx_RNN_epsilon_0.1_62.5k_mean,sim,murcko,0.858844,-,0.055549,0.686277,1.000000
5,DrugEx_RNN_epsilon_0.6_62.5k_mean,sim,murcko,0.905471,-,0.711855,0.793160,0.598719
6,GB_GA_mut_r_0.01_62.5k_mean,sim,murcko,0.480449,-,0.000000,0.450879,0.122956
7,GB_GA_mut_r_0.5_62.5k_mean,sim,murcko,0.000000,-,0.003107,0.423469,0.209230
8,addcarbon_62.5k_mean,sim,murcko,0.986947,-,0.197500,0.000000,0.030459
